# Scrape the current data


In [7]:
from hn_jobs_chat.var import url, postsTableName


'https://news.ycombinator.com/item?id=41129813'

In [5]:
from hn_jobs_chat.db import connectToDB

query = """CREATE TABLE IF NOT EXISTS """ + postsTableName + """ (
        id SERIAL PRIMARY KEY, 
        hnuser TEXT, 
        date TIMESTAMP, 
        comment TEXT,
        fileName TEXT,
        html TEXT
        );"""

conn, cursor = connectToDB()

cursor.execute(query)

conn.commit()   
conn.close() 

In [6]:
from hn_jobs_chat.scraper import scrape

# April 2024
monthPosts = scrape(url)
monthPosts = monthPosts[0:154]

Scraping page 1
all pages scraped
Scraped 181 posts


In [7]:
import psycopg2

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cursor = conn.cursor()

for post in monthPosts:
    hnuser = post["hnuser"]
    date = post["date"]
    comment = post["comment"]
    
    fileName = post["fileName"]
    html = post["html"]

    cursor.execute(
        """INSERT INTO """ + postsTableName + 
        """ (hnuser, date, comment, fileName, html) VALUES (%s, %s, %s, %s, %s)"""
        , (hnuser, date, comment, fileName, html)
    )

    conn.commit()

conn.close()


Since it's small, let's just back it up to a csv file, in case something get's messed up later.

In [ ]:
from hn_jobs_chat import init_scrape
import psycopg2

postsTableName = init_scrape.postsTableName

conn = psycopg2.connect("dbname=Bumpant user=Bumpant password=ampegskb")
cursor = conn.cursor()

query = """SELECT * FROM """ + postsTableName

cursor.execute(query)

data = cursor.fetchall()

import csv

with open('backup_after_parse' + postsTableName + '.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([i[0] for i in cursor.description])  # write headers
    writer.writerows(data)

conn.commit()   
conn.close() 
# Lets start by backing up the table <<postsTableName>> to a csv file

In [8]:
#| hide
from nbdev import nbdev_export
nbdev_export()